<a href="https://colab.research.google.com/github/srehaag/legal_info_tech_w25/blob/main/Nazdona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install datasets -q

# Load dataset
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("refugee-law-lab/canadian-legal-data",
                       split="train",
                       data_dir="RLLR")
df = pd.DataFrame(dataset)

# Convert "Other" col (in JSON) to cols
import json
df = df.join(df.other.apply(json.loads).apply(pd.Series))
df = df.drop(columns=["other"])

# display first 5 rows
df.head(5)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,country,case_type,member
0,2023 RLLR 120,VC3-09637,RLLR,2023,,en,12/20/2023,https://refugeelab.ca/rllr/2023rllr120,2025-01-05,2023 RLLR 120\n\nCitation: 2023 RLLR 120\nTrib...,Cameroon,Political Opinion,Kate Bilkevitch
1,2023 RLLR 119,VC3-07126,RLLR,2023,,en,12/19/2023,https://refugeelab.ca/rllr/2023rllr119,2025-01-05,2023 RLLR 119\n\nCitation: 2023 RLLR 119\nTrib...,Russia,Political Opinion,Hannah Gray
2,2023 RLLR 118,VC3-06815,RLLR,2023,,en,10/31/2023,https://refugeelab.ca/rllr/2023rllr118,2025-01-05,2023 RLLR 118\n\nCitation: 2023 RLLR 118\nTrib...,Lebanon,PSG: SOGIE,David Jones
3,2023 RLLR 117,VC3-06662,RLLR,2023,,en,11/2/2023,https://refugeelab.ca/rllr/2023rllr117,2025-01-05,2023 RLLR 117\n\nCitation: 2023 RLLR 117\nTrib...,USA,PSG: SOGIE,Sandeep Chauhan
4,2023 RLLR 116,VC3-04003,RLLR,2023,,en,6/26/2023,https://refugeelab.ca/rllr/2023rllr116,2025-01-05,2023 RLLR 116\n\nCitation: 2023 RLLR 116\nTrib...,Ukraine,PSG: SOGIE,Hannah Gray


In [2]:
# set up openai

from openai import OpenAI

# get openai key from colab
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# set up client
client = OpenAI()

# set up get_completion function
def get_completion(user_message,
        system_message="You are a helpful assistant to a Canadian law student",
        model = "gpt-4o",
        temperature = 0):

    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
    )
    return completion.choices[0].message.content

# test system
question = "Is Roncarelli still good law?"
print(get_completion(question))

Yes, Roncarelli v. Duplessis, [1959] S.C.R. 121, remains a foundational case in Canadian administrative law. It is still considered good law and is frequently cited for its principles regarding the rule of law and the limits of discretionary power. The case established that public officials must exercise their discretion within the bounds of the law and cannot act arbitrarily or for improper purposes. Roncarelli v. Duplessis is often referenced in discussions about the abuse of power and the requirement for decision-makers to act in good faith and for lawful purposes.


In [3]:
# setup process to create prompt

system_message = """You are an assistant to a refugee law lawyer.
You are given instructions and a text, and you extract the requested
information from the text. You only return the requested information,
without any explanation. If the answer is unclear, you rertun only
the text string: unclear"""

def generate_user_prompt(task, document):
  prefix = "INTRUCTIONS: " + task + "\n DOCUMENT: "
  suffix = "\n ANSWER: "
  return prefix + document + suffix

In [4]:
# apply to df

def get_country(document):
  task = "Extract the country of the claimant"
  user_message = generate_user_prompt(task, document)
  return get_completion(user_message=user_message, system_message=system_message)

# filter df to get first 5 cases to reduce costs for test
df = df.head(5)

# apply get_country to df and put in new col: GENAI_country
df["GENAI_country"] = df.unofficial_text.apply(get_country)
df.head(5)

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,country,case_type,member,GENAI_country
0,2023 RLLR 120,VC3-09637,RLLR,2023,,en,12/20/2023,https://refugeelab.ca/rllr/2023rllr120,2025-01-05,2023 RLLR 120\n\nCitation: 2023 RLLR 120\nTrib...,Cameroon,Political Opinion,Kate Bilkevitch,Cameroon
1,2023 RLLR 119,VC3-07126,RLLR,2023,,en,12/19/2023,https://refugeelab.ca/rllr/2023rllr119,2025-01-05,2023 RLLR 119\n\nCitation: 2023 RLLR 119\nTrib...,Russia,Political Opinion,Hannah Gray,Russia
2,2023 RLLR 118,VC3-06815,RLLR,2023,,en,10/31/2023,https://refugeelab.ca/rllr/2023rllr118,2025-01-05,2023 RLLR 118\n\nCitation: 2023 RLLR 118\nTrib...,Lebanon,PSG: SOGIE,David Jones,Lebanon
3,2023 RLLR 117,VC3-06662,RLLR,2023,,en,11/2/2023,https://refugeelab.ca/rllr/2023rllr117,2025-01-05,2023 RLLR 117\n\nCitation: 2023 RLLR 117\nTrib...,USA,PSG: SOGIE,Sandeep Chauhan,United States of America (USA)
4,2023 RLLR 116,VC3-04003,RLLR,2023,,en,6/26/2023,https://refugeelab.ca/rllr/2023rllr116,2025-01-05,2023 RLLR 116\n\nCitation: 2023 RLLR 116\nTrib...,Ukraine,PSG: SOGIE,Hannah Gray,Ukraine


In [6]:
# example of asking different question

def get_gender(document):
  task = "Extract the gender of the principal claimant. Retrun only: male, female, other"
  user_message = generate_user_prompt(task, document)
  return get_completion(user_message=user_message, system_message=system_message)

# filter df to get first 5 cases to reduce costs for test
df = df.head(5)

# apply get_gender to df and put in new col: GENAI_gender
df["GENAI_gender"] = df.unofficial_text.apply(get_gender)
df.head(5)

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,country,case_type,member,GENAI_country,GENAI_gender
0,2023 RLLR 120,VC3-09637,RLLR,2023,,en,12/20/2023,https://refugeelab.ca/rllr/2023rllr120,2025-01-05,2023 RLLR 120\n\nCitation: 2023 RLLR 120\nTrib...,Cameroon,Political Opinion,Kate Bilkevitch,Cameroon,male
1,2023 RLLR 119,VC3-07126,RLLR,2023,,en,12/19/2023,https://refugeelab.ca/rllr/2023rllr119,2025-01-05,2023 RLLR 119\n\nCitation: 2023 RLLR 119\nTrib...,Russia,Political Opinion,Hannah Gray,Russia,male
2,2023 RLLR 118,VC3-06815,RLLR,2023,,en,10/31/2023,https://refugeelab.ca/rllr/2023rllr118,2025-01-05,2023 RLLR 118\n\nCitation: 2023 RLLR 118\nTrib...,Lebanon,PSG: SOGIE,David Jones,Lebanon,female
3,2023 RLLR 117,VC3-06662,RLLR,2023,,en,11/2/2023,https://refugeelab.ca/rllr/2023rllr117,2025-01-05,2023 RLLR 117\n\nCitation: 2023 RLLR 117\nTrib...,USA,PSG: SOGIE,Sandeep Chauhan,United States of America (USA),other
4,2023 RLLR 116,VC3-04003,RLLR,2023,,en,6/26/2023,https://refugeelab.ca/rllr/2023rllr116,2025-01-05,2023 RLLR 116\n\nCitation: 2023 RLLR 116\nTrib...,Ukraine,PSG: SOGIE,Hannah Gray,Ukraine,female
